# Simple EV Controller 

In [3]:
from tdshub.gridlabd import Gridlabd
from pymongo import MongoClient
import datetime

def init_db():
    """ Connnect to the mongo db"""
    connection = MongoClient("localhost", 27017)
    handle = connection["gridlabd"]
    return handle

mongo = init_db()

def insert(self):
    mongo.exp.insert_one({'query': self.config(),
                          'config': self.export(),
                          'graph': self.graph(),
                          'data': self.results(),
                          'timestamp': str(datetime.datetime.now())}) 
    return self

Gridlabd.insert = insert

Clear the database...

In [4]:
mongo.exp.delete_many({});

## The experiments

### Ev controllers

In [5]:
b = Gridlabd({'evs': {'number': 40, 'controller': 'tou'},
              'houses': {'number': 40},
              'solar': {'number': 40 },
              'name': "ev ctlr and solar",
              'end_time': '2013-07-02 08:00:00' })

In [6]:
df = b.execute().df()

(df.vmask('object', 'triplex_meter')
       .vmask('attr_group', 'power')
       .pivot(index="timestamp", columns="name", values='value')
       .sum(axis=1)).plotly()

executed in 18.6840839386 seconds, (346957, 7)


In [7]:
b.insert()

### Evs and solar, no controllers. 

In [28]:
b = Gridlabd({'evs': {'number': 40},
              'houses': {'number': 40},
              'solar': {'number': 40 },
              'name': "ev and solar",
              'end_time': '2013-07-02 08:00:00' })

In [29]:
df = b.execute().df()

(df.vmask('object', 'triplex_meter')
       .vmask('attr_group', 'power')
       .pivot(index="timestamp", columns="name", values='value')
       .sum(axis=1)).plotly()

executed in 23.1947720051 seconds, (346957, 7)


In [30]:
b.insert()

### Solar

In [31]:
b = Gridlabd({'houses': {'number': 40},
              'solar': {'number': 40 },
              'name': "solar",
              'end_time': '2013-07-02 08:00:00' })

In [32]:
df = b.execute().df()

(df.vmask('object', 'triplex_meter')
       .vmask('attr_group', 'power')
       .pivot(index="timestamp", columns="name", values='value')
       .sum(axis=1)).plotly()

executed in 18.6646299362 seconds, (218517, 7)


In [33]:
b.insert()

### Houses only

In [34]:
b = Gridlabd({'houses': {'number': 40},
              'name': "houses",
              'end_time': '2013-07-02 08:00:00' })

In [35]:
df = b.execute().df()

(df.vmask('object', 'triplex_meter')
       .vmask('attr_group', 'power')
       .pivot(index="timestamp", columns="name", values='value')
       .sum(axis=1)).plotly()

executed in 15.0968530178 seconds, (218517, 7)


In [36]:
b.insert()

### 40 houses example

In [15]:
from tdshub.gridlabd import Gridlabd

In [16]:
b = Gridlabd().houses(40).end_time('2013-07-02 08:00:00')

In [17]:
df = b.execute().df()

executed in 12.3077287674 seconds, (218686, 7)


In [18]:
power = (df.vmask('object', 'triplex_meter')
       .vmask('attr_group', 'power')
       .pivot(index="timestamp", columns="name", values='value'))

power.sum(axis=1).plotly()

In [19]:
setpoint = (df.vmask('attr_group', 'cooling_setpoint')
              .pivot(index="timestamp", columns="name", values='value'))

setpoint.plotly()

In [20]:
b.plot()

In [16]:
from bson import ObjectId
import pandas as pd

cursor = mongo.exp.find({'_id': ObjectId("56fff6b6ccc818531edb14b0")})
cursor[0].keys()

[u'graph', u'data', u'timestamp', u'query', u'_id', u'config']

In [19]:
def get_df():
    _data = pd.DataFrame(cursor[0]['data'])
    _id = cursor[0]['_id']
    _data.set_index("timestamp", inplace=True)
    _data = _data.unstack()
    _data = _data.reset_index()
    _data.columns = ["id", "x", "value"]
    _data['object'] = _data['id'].map(lambda x: x.split(":")[0])
    _data['attribute'] =  _data['id'].map(lambda x: x.split(":")[1])
    _data['name'] =  _data['id'].map(lambda x: x.split(":")[2])
    _data['attr_group'] =  _data['id'].map(lambda x: x.split(":")[3])
    _data['house_group'] =  _data['id'].map(lambda x: x.split(":")[4])
    _data['id'] = _id
    return _data[['x', 'id', 'object', 'attribute', 'name', 'attr_group', 'house_group', 'value']]

# print(cursor[0]['data'])
df = get_df()
# print(df.head())


df = df.mask('object', 'house')

df = df.mask('attr_group', 'power')

print(df.head())
print(df.tail())
print(df.columns)
print("----------------------")

                        x                        id object   attribute  \
67600 2013-07-01 04:00:00  56fff6b6ccc818531edb14b0  house  hvac_power   
67601 2013-07-01 04:10:00  56fff6b6ccc818531edb14b0  house  hvac_power   
67602 2013-07-01 04:20:00  56fff6b6ccc818531edb14b0  house  hvac_power   
67603 2013-07-01 04:30:00  56fff6b6ccc818531edb14b0  house  hvac_power   
67604 2013-07-01 04:40:00  56fff6b6ccc818531edb14b0  house  hvac_power   

          name attr_group house_group value  
67600  house_0      power     house_0     0  
67601  house_0      power     house_0     0  
67602  house_0      power     house_0     0  
67603  house_0      power     house_0     0  
67604  house_0      power     house_0     0  
                        x                        id object   attribute  \
74355 2013-07-02 07:20:00  56fff6b6ccc818531edb14b0  house  hvac_power   
74356 2013-07-02 07:30:00  56fff6b6ccc818531edb14b0  house  hvac_power   
74357 2013-07-02 07:40:00  56fff6b6ccc818531edb14b0  hous

In [27]:
df['attribute'].unique()

array([u'hvac_power'], dtype=object)

In [25]:
df.pivot_table(index="x", columns="name", values="value", aggfunc='sum')


name,house_0,house_1,house_10,house_11,house_12,house_13,house_14,house_15,house_16,house_17,...,house_36,house_37,house_38,house_39,house_4,house_5,house_6,house_7,house_8,house_9
x,,,,,,,,,,,,,,,,,,,,,
2013-07-01 04:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-01 04:10:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-01 04:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-01 04:30:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-01 04:40:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-01 04:50:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-01 05:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-01 05:10:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-01 05:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
